In [ ]:
# Import Libraries
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                                    Dropout, BatchNormalization, RandomFlip,
                                    RandomRotation, GlobalAveragePooling2D)
from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (accuracy_score, classification_report,
                            confusion_matrix, roc_curve, auc)
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Enable mixed precision training
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/Dataset"

In [ ]:
print (os.listdir(DATASET_PATH))


['IQ-OTH_NCCD lung cancer dataset.txt', 'Bengin', 'Malignant', 'Normal', 'best_ResNet50.keras', 'best_DenseNet121.keras', 'best_EnhancedCNN.keras']


In [ ]:
CLASSES = ["Bengin", "Malignant", "Normal"]
CLASS_LABELS = {cls: i for i, cls in enumerate(CLASSES)}
IMG_SIZE = (64, 64)

In [ ]:
# Image loading function
def load_images(dataset_path, img_size):
    images, labels = [], []
    for category in CLASSES:
        category_path = os.path.join(dataset_path, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, img_size) / 255.0
                images.append(img_resized)
                labels.append(CLASS_LABELS[category])
    return np.array(images), np.array(labels)

In [ ]:
# Image loading function
def load_images(dataset_path, img_size):
    images, labels = [], []
    for category in CLASSES:
        category_path = os.path.join(dataset_path, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, img_size) / 255.0
                images.append(img_resized)
                labels.append(CLASS_LABELS[category])
    return np.array(images), np.array(labels)

In [ ]:
X, y = load_images(DATASET_PATH, IMG_SIZE)
X = np.expand_dims(X, axis=-1)
X = np.repeat(X, 3, axis=-1)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=42, stratify=y)

In [ ]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(
    X_train.reshape(X_train.shape[0], -1), y_train)
X_train_resampled = X_train_resampled.reshape(-1, IMG_SIZE[0], IMG_SIZE[1], 3)

In [ ]:
# Convert labels
y_train_resampled = to_categorical(y_train_resampled, num_classes=len(CLASSES))
y_test = to_categorical(y_test, num_classes=len(CLASSES))

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# For validation/test data, we only rescale
test_datagen = ImageDataGenerator()

# Create generators
train_generator = train_datagen.flow(
    X_train_resampled,
    y_train_resampled,
    batch_size=32
)

val_generator = test_datagen.flow(
    X_test,
    y_test,
    batch_size=32
)

In [ ]:
# Enhanced CNN Model
def build_enhanced_cnn():
    model = Sequential([
        # Data augmentation
        RandomFlip("horizontal"),
        RandomRotation(0.1),

        # Feature extraction
        Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        Dropout(0.3),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),

        # Classification
        GlobalAveragePooling2D(),
        Dense(128, activation='relu', kernel_regularizer='l2'),
        Dropout(0.5),
        Dense(len(CLASSES), activation='softmax', dtype='float32')
    ])
    model.compile(optimizer=Adam(0.0001),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [ ]:
# Transfer Learning Model
def create_transfer_model(base_model):
    # Freeze base model layers
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(len(CLASSES), activation='softmax', dtype='float32')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(1e-4),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [ ]:
# Initialize models
resnet = create_transfer_model(ResNet50(weights='imagenet', include_top=False, input_shape=(64,64,3)))
densenet = create_transfer_model(DenseNet121(weights='imagenet', include_top=False, input_shape=(64,64,3)))
enhanced_cnn = build_enhanced_cnn()

models = [resnet, densenet, enhanced_cnn]
model_names = ['ResNet50', 'DenseNet121', 'EnhancedCNN']

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Improved plotting function that handles both History objects and dictionaries
def plot_training(history, name):
    plt.figure(figsize=(12, 5))

    # Handle both History objects and raw dictionaries
    if hasattr(history, 'history'):  # If it's a History object
        history_dict = history.history
    else:  # If it's already a dictionary
        history_dict = history

    # Check available metrics
    print(f"\nAvailable metrics for {name}:", history_dict.keys())

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    if 'accuracy' in history_dict:
        plt.plot(history_dict['accuracy'], label='Train Accuracy')
    if 'val_accuracy' in history_dict:
        plt.plot(history_dict['val_accuracy'], label='Val Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title(f'{name} - Accuracy')
    plt.legend()

    # Loss Plot
    plt.subplot(1, 2, 2)
    if 'loss' in history_dict:
        plt.plot(history_dict['loss'], label='Train Loss')
    if 'val_loss' in history_dict:
        plt.plot(history_dict['val_loss'], label='Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'{name} - Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# ## Ensemble Learning & Evaluation
def ensemble_predict(models, x_input):
    predictions = [model.predict(x_input) for model in models]
    avg_prediction = np.mean(predictions, axis=0)
    return np.argmax(avg_prediction, axis=1)

In [ ]:
# Generate predictions
y_pred_ensemble = ensemble_predict(models, X_test)
y_test_labels = np.argmax(y_test, axis=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 707ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 766ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step


In [ ]:
# Enhanced evaluation metrics
def plot_metrics(y_true, y_pred, title):
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))

    # Classification Report
    report = classification_report(y_true, y_pred, target_names=CLASSES, output_dict=True)
    report_df = pd.DataFrame(report).iloc[:-1, :].T
    sns.heatmap(report_df, annot=True, cmap="Blues", ax=axes[0])
    axes[0].set_title(f"{title} - Classification Report")

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Oranges",
                xticklabels=CLASSES, yticklabels=CLASSES, ax=axes[1])
    axes[1].set_title(f"{title} - Confusion Matrix")

    # ROC Curve
    y_prob = np.mean([model.predict(X_test) for model in models], axis=0)
    fpr, tpr, _ = roc_curve(y_test, y_prob, multi_class="ovr")
    roc_auc = auc(fpr, tpr)
    axes[2].plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    axes[2].set_title("ROC Curve")
    axes[2].legend(loc="lower right")

    plt.tight_layout()
    plt.show()

In [ ]:


def plot_metrics(y_true, y_pred, model_name="Model", class_names=None):
    """
    Enhanced version that handles multi-class classification

    Parameters:
    - y_true: True labels (1D array)
    - y_pred: Predicted labels (1D array)
    - model_name: Name for title
    - class_names: List of class names for display
    """
    # Convert to numpy arrays if needed
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # If class_names not provided, use numbers
    if class_names is None:
        class_names = [f"Class {i}" for i in range(len(np.unique(y_true)))]

    # Classification report
    print(f"\nClassification Report for {model_name}:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# FIRST define your class names (example for 3 classes)
class_names = ["Normal", "Benign", "Malignant"]  # Replace with your actual class names

# THEN call the function
print("Ensemble Model Evaluation:")
plot_metrics(y_test_labels, y_pred_ensemble, "Ensemble Model", class_names)

# Alternative if you don't have names - will auto-generate Class 0, Class 1, etc.
# plot_metrics(y_test_labels, y_pred_ensemble, "Ensemble Model")

In [ ]:

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle

# 1. First get your predicted probabilities
# For Keras/TensorFlow models:
y_pred_prob = model.predict(X_test)  # This gives probabilities for each class

# For scikit-learn models:
# y_pred_prob = model.predict_proba(X_test)

# 2. Define your class names (replace with your actual class names)
class_names = ["Class0", "Class1", "Class2"]  # Example for 3 classes

# 3. The plotting function
def plot_multiclass_roc_auc(y_true, y_pred_prob, class_names):
    """
    Plot ROC curves for multi-class classification

    Parameters:
    y_true: True labels (1D array of class indices)
    y_pred_prob: Predicted probabilities (2D array [n_samples, n_classes])
    class_names: List of class names
    """
    n_classes = len(class_names)

    # Binarize the true labels
    y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

    # Compute ROC curve and AUC for each class
    fpr, tpr, roc_auc = dict(), dict(), dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_prob.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Plot all ROC curves
    plt.figure(figsize=(10, 8))
    colors = cycle(['blue', 'red', 'green', 'yellow', 'purple'])

    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='{0} (AUC = {1:0.2f})'.format(class_names[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

# 4. Verify your inputs
print("True labels shape:", y_test_labels.shape)
print("Predicted probabilities shape:", y_pred_prob.shape)
print("Class names:", class_names)

# 5. Generate the plot
plot_multiclass_roc_auc(y_test_labels, y_pred_prob, class_names)

In [ ]:
# Generate meta features
train_preds = [model.predict(X_train_resampled) for model in models]
test_preds = [model.predict(X_test) for model in models]

X_train_meta = np.hstack(train_preds)
X_test_meta = np.hstack(test_preds)


In [ ]:
# Train meta-model
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(X_train_meta, np.argmax(y_train_resampled, axis=1))

In [ ]:
# Evaluate stacking
y_pred_stacking = meta_model.predict(X_test_meta)
print("\nStacking Model Evaluation:")
plot_metrics(y_test_labels, y_pred_stacking, "Stacking Model")


In [ ]:
# ## Model Interpretation (Grad-CAM)
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    return heatmap / np.max(heatmap)

In [ ]:
# Visualization function
def visualize_gradcam(model, image, last_conv_layer_name):
    heatmap = make_gradcam_heatmap(np.expand_dims(image, 0), model, last_conv_layer_name)
    plt.matshow(heatmap)
    plt.show()

In [ ]:
def visualize_gradcam(model, img_array, layer_name, alpha=0.4):
    """Visualize GradCAM heatmap for a given model and layer"""

    # 1. Preprocess the image
    if len(img_array.shape) == 3:
        img_array = np.expand_dims(img_array, axis=0)
    img_tensor = tf.convert_to_tensor(img_array)

    # 2. Create gradient model
    grad_model = Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    # 3. Compute gradients
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    # 4. Get gradients and generate heatmap
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # 5. Process heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = heatmap.numpy()

    # 6. Resize heatmap to match image
    heatmap = tf.image.resize(
        heatmap[..., tf.newaxis],
        (img_array.shape[1], img_array.shape[2])
    ).numpy().squeeze()

    # 7. Display results
    plt.figure(figsize=(10, 5))

    # Original image
    plt.subplot(1, 2, 1)
    plt.imshow(img_array[0])  # Remove batch dimension
    plt.title("Original")
    plt.axis('off')

    # Heatmap
    plt.subplot(1, 2, 2)
    plt.imshow(heatmap, cmap='viridis')
    plt.title("GradCAM Heatmap")
    plt.axis('off')

    plt.show()

    # Overlay
    plt.figure(figsize=(6, 6))
    plt.imshow(img_array[0])
    plt.imshow(heatmap, cmap='viridis', alpha=alpha)
    plt.title("GradCAM Overlay")
    plt.axis('off')
    plt.show()

# Example usage
sample_image = X_test[0]  # Ensure this is a single image with shape (H, W, C)
visualize_gradcam(resnet, sample_image, "conv5_block3_out")

In [ ]:
def plot_accuracy_comparison(models, model_names, X_test, y_test):
    plt.figure(figsize=(12, 6))
    for model, name in zip(models, model_names):
        y_pred = np.argmax(model.predict(X_test), axis=1)
        acc = accuracy_score(y_test_labels, y_pred)
        plt.bar(name, acc, alpha=0.6)
    plt.title("Model Accuracy Comparison")
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)
    plt.show()

plot_accuracy_comparison(models, model_names, X_test, y_test_labels)

In [ ]:
# ## Corrected Interactive Prediction with Working Grad-CAM and Automatic Best Model Selection
def predict_single_image():
    # Load models with error handling
    try:
        models = {
            'ResNet50': tf.keras.models.load_model("/content/drive/MyDrive/Dataset/best_ResNet50.keras"),
            'DenseNet121': tf.keras.models.load_model("/content/drive/MyDrive/Dataset/best_DenseNet121.keras"),
            'EnhancedCNN': tf.keras.models.load_model("/content/drive/MyDrive/Dataset/best_EnhancedCNN.keras")
        }
        # Model performance metrics from your training
        model_performance = {
            'ResNet50': {'val_acc': 43.23, 'color': 'blue'},
            'DenseNet121': {'val_acc': 71.35, 'color': 'red'},
            'EnhancedCNN': {'val_acc': 66.67, 'color': 'green'}
        }
        best_model_name = max(model_performance.items(), key=lambda x: x[1]['val_acc'])[0]
    except Exception as e:
        print(f"❌ Error loading models: {str(e)}")
        return

    # Get image path
    while True:
        image_path = input("\n📁 Enter CT scan image path (or 'q' to quit): ").strip()
        if image_path.lower() == 'q':
            return

        try:
            # Read and validate image
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                raise ValueError("Invalid image or path")

            # Preprocessing pipeline
            img_resized = cv2.resize(img, IMG_SIZE)
            img_normalized = img_resized / 255.0
            img_3channel = np.repeat(img_normalized[..., np.newaxis], 3, axis=-1)
            img_input = np.expand_dims(img_3channel, axis=0)

            # Get predictions from all models
            preds = {name: model.predict(img_input, verbose=0)[0] for name, model in models.items()}

            # Get best model prediction
            best_pred = preds[best_model_name]
            best_pred_class = CLASSES[np.argmax(best_pred)]
            best_confidence = np.max(best_pred) * 100

            # Calculate ensemble prediction (average of all models)
            avg_pred = np.mean(list(preds.values()), axis=0)
            ensemble_class = CLASSES[np.argmax(avg_pred)]
            ensemble_confidence = np.max(avg_pred) * 100

            # Create visualization figure
            plt.figure(figsize=(20, 8))

            # 1. Original Image
            plt.subplot(2, 3, 1)
            plt.imshow(img_resized, cmap='gray')
            plt.title(f"Original CT Scan\n{os.path.basename(image_path)}", pad=20)
            plt.axis('off')

            # 2. Grad-CAM Heatmap from Best Model
            plt.subplot(2, 3, 2)
            try:
                # Generate heatmap using best model
                layer_name = "conv5_block3_out" if best_model_name == "ResNet50" else "conv5_block3_3_conv" if best_model_name == "DenseNet121" else "conv2d_3"
                heatmap = make_gradcam_heatmap(img_input, models[best_model_name], layer_name)

                # Resize heatmap to match original image
                heatmap = cv2.resize(heatmap, (img_resized.shape[1], img_resized.shape[0]))

                # Display overlay
                plt.imshow(img_resized, cmap='gray')
                plt.imshow(heatmap, cmap='jet', alpha=0.5)
                plt.title(f"Attention Heatmap ({best_model_name})", pad=20)
                plt.colorbar(label='Attention Intensity')
                plt.axis('off')
            except Exception as e:
                print(f"⚠️ Heatmap generation failed: {str(e)}")
                plt.clf()
                plt.text(0.5, 0.5, "Heatmap Unavailable", ha='center', va='center')
                plt.axis('off')

            # 3. Best Model Prediction
            plt.subplot(2, 3, 3)
            colors = ['#4CAF50' if i == np.argmax(best_pred) else '#607D8B' for i in range(len(CLASSES))]
            bars = plt.barh(CLASSES, best_pred * 100, color=colors)
            plt.bar_label(bars, fmt='%.2f%%', padding=5)
            plt.xlim(0, 100)
            plt.title(f"Best Model ({best_model_name})\nPrediction: {best_pred_class}\nConfidence: {best_confidence:.2f}%", pad=20)
            plt.xlabel("Confidence (%)")
            plt.grid(axis='x', alpha=0.3)

            # 4. Ensemble Prediction
            plt.subplot(2, 3, 4)
            colors = ['#4CAF50' if i == np.argmax(avg_pred) else '#607D8B' for i in range(len(CLASSES))]
            bars = plt.barh(CLASSES, avg_pred * 100, color=colors)
            plt.bar_label(bars, fmt='%.2f%%', padding=5)
            plt.xlim(0, 100)
            plt.title(f"Ensemble Prediction\nPrediction: {ensemble_class}\nConfidence: {ensemble_confidence:.2f}%", pad=20)
            plt.xlabel("Confidence (%)")
            plt.grid(axis='x', alpha=0.3)

            # 5. Model Comparison
            plt.subplot(2, 3, 5)
            model_names = list(model_performance.keys())
            accuracies = [model_performance[name]['val_acc'] for name in model_names]
            colors = [model_performance[name]['color'] for name in model_names]

            # Highlight best model
            for i, name in enumerate(model_names):
                if name == best_model_name:
                    plt.bar(i, accuracies[i], color=colors[i], edgecolor='gold', linewidth=3)
                else:
                    plt.bar(i, accuracies[i], color=colors[i])

            plt.xticks(range(len(model_names)), model_names)
            plt.ylabel("Validation Accuracy (%)")
            plt.title("Model Performance Comparison", pad=20)
            plt.ylim(0, 100)
            plt.grid(axis='y', alpha=0.3)

            plt.tight_layout()
            plt.show()
            return

        except Exception as e:
            print(f"❌ Error: {str(e)}")

# Run the interface
print("\n" + "="*50)
print("🩺 LUNG CANCER CLASSIFICATION SYSTEM")
print("="*50)
print(f"Available classes: {CLASSES}\n")

while True:
    predict_single_image()
    cont = input("\n🔍 Analyze another image? (y/n): ").lower()
    if cont != 'y':
        print("👋 Exiting system...")
        break


🩺 LUNG CANCER CLASSIFICATION SYSTEM


NameError: name 'CLASSES' is not defined